# Brechung von Licht

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

In diesem Jupyter-Notebook wird der Weg eines Lichtstrahls in einem Medium mit räumlich veränderlichem Brechungsindex berechnet. Dazu wird das Fermat'sche Prinzip verwendet, gemäß dem sich Licht zwischen zwei Punkten so ausbreitet, dass die optische Weglänge minimal wird. Wir betrachten die Lichtausbreitung in der $x$-$y$-Ebene, wobei der Brechungsindex von $y$ abhängen kann. Dann ist die optische Weglänge durch

$$\int_{x_0}^{x_1}\text{d}x\,n(y)\sqrt{1+\left(\frac{\text{d}y}{\text{d}x}\right)^2}$$

gegeben.

Im ersten Teil des Jupyter-Notebooks betrachten wir den Fall, in dem der Brechungsindex an der Grenzfläche $y=0$ einen Sprung aufweist und ansonsten konstant ist. Numerisch werden wir dieses Problem in Form einer Optimierungsaufgabe lösen und das Ergebnis mit der analytischen Lösung, die durch das Snellius'sche Brechungsgesetz gegeben ist, vergleichen.

Im zweiten Teil wird zugelassen, dass sich der Brechungsindex $n(y)$ kontinuierlich als Funktion der $y$-Koordinate ändert. Auch hierfür werden wir zunächst ein Optimierungsproblem numerisch lösen, um die Form des Lichtstrahls zu berechnen.

Als Alternative hierzu wird im letzten Teil die zugehörige Euler-Lagrange-Gleichung als Randwertproblem gelöst. Dazu fasst man den obigen Integranden als Lagrangefunktion auf und erhält somit die Differentialgleichung

$$y'' = \frac{1+(y')^2}{n(y)}\frac{\text{d}n}{\text{d}y}\,,$$

die zusammen mit Anfangs- und Endpunkt ein Randwertproblem definiert.

### Importanweisungen

Neben bereits bekannten Importanweisungen wird hier der Namensraum des Moduls `optimize` aus dem SciPy-Paket importiert, um ein Minimalisierungsproblem lösen zu können.

In [ ]:
from functools import partial
import numpy as np
import numpy.linalg as LA
from scipy import integrate, optimize
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt

plt.style.use("numphyspy.style")

## Snellius'sches Brechungsgesetz

Es wird die Brechung an einer Grenzfläche bei $y=0$ betrachtet, die einen optisch dichteren Bereich $y<0$ mit Brechungsindex $n$ von einem Bereich $y>0$ mit Brechungsindex 1 trennt. Der Lichtstrahl soll vom Punkt $(0\vert-1)$ zum Punkt $(x_0\vert 1)$ laufen. Im Rahmen des Optimierungsproblems wird die Koordinate $x$ berechnet, an der der Lichtstrahl die Grenzfläche durchstößt. Daraus ergeben sich die Winkel $\alpha_1$ und $\alpha_2$ bezüglich der Normalen auf die Grenzfläche, die durch das Snellius'sche Brechungsgesetz

$$n\sin(\alpha_1) = \sin(\alpha_2)$$

verknüpft sind.

### Berechnung der optischen Weglänge

Die optische Weglänge ergibt sich aus zwei Wegstücken:
- Weg von $(0\vert-1)$ nach $(x\vert 0)$ mit Brechungsindex $n$
- Weg von $(x\vert 0)$ nach $(x_0\vert 1)$ mit Brechungsindex 1

In [ ]:
def optical_path_length_1(x, n, x_0):
    return n*np.hypot(1, x) + np.hypot(1, x_0-x)

### Minimierung der optischen Weglänge

Um unterschiedliche Brechungswinkel betrachten zu können, soll die Endkoordinate $x_0$ des Lichtsstrahls `n_points` gleichmäßig verteilte Werte zwischen 0 und `x_0_max` annehmen. Der Brechungsindex im unteren Halbraum wird durch das Argument `n` übergeben.

Für jeden einzelnen Wert von $x_0$ wird das Optimierungsproblem gelöst und die optimale Koordinate $x$ für den Durchstoßpunkt durch die Grenzfläche bestimmt. Zum Vergleich mit dem Snellius'schen Brechungsgesetz werden abschließend die Winkel $\alpha_1$ und $\alpha_2$ berechnet und zurückgegeben.

In [ ]:
def shortest_path_1(n_points, n, x_0_max):
    x_values = np.zeros(n_points)
    x_0_values = np.linspace(0, x_0_max, n_points)
    for idx, x_0 in enumerate(x_0_values):
        x_init = np.array([x_0/2])
        opt_result = optimize.minimize(
            optical_path_length_1, x_init, args=(n, x_0))
        x_values[idx] = opt_result.x[0]
    alpha_1 = np.arctan(x_values)
    alpha_2 = np.arctan(x_0_values-x_values)
    return alpha_1, alpha_2

### Snellius'sches Brechungsgesetz

Zum Vergleich mit den numerisch erhaltenen Ergebnissen werden $\alpha_1$ und $\alpha_2$ für einen Brechungsindex `n` in der unteren Halbebene berechnet. Dabei läuft `alpha_1` bis knapp unter den Grenzwinkel $\arcsin(1/n)$ der Totalreflexion.

In [ ]:
def snellius_law(n):
    alpha_1 = np.linspace(0, np.arcsin(1/n)-1e-7, 200)
    alpha_2 = np.arcsin(n*np.sin(alpha_1))
    return alpha_1, alpha_2

### Implementierung der Bedienelemente und Darstellung der Brechungswinkel

Mit Hilfe der Schieberegler lassen sich die folgenden Parameter einstellen:
- `n_points`: Anzahl der Endpunkte
- `n`: Brechungsindex im unteren Halbraum
- `x_0_max`: Maximalwert der $x$-Koordinate des Endpunkts

Die Abhängigkeit $\alpha_2(\alpha_1)$ wird graphisch dargestellt. Dabei ergeben sich die Punkte aus der Lösung des Optimierungsproblems und die durchgezogene Kurve ergibt sich aus dem Snellius'schen Brechungsgesetz.

In [ ]:
widget_dict = {"n_points":
               widgets.IntSlider(
                   value=10, min=1, max=20, step=1,
                   description=r"$n_\text{Punkte}$"),
               "n":
               widgets.FloatSlider(
                   value=1.4, min=1, max=2, step=0.01,
                   description="$n$"),
               "x_0_max":
               widgets.FloatSlider(
                   value=8, min=0, max=20, step=0.1,
                   description="$x_0$")
               }

@interact(**widget_dict)
def plot_result_snellius(n_points, n, x_0_max):
    alpha_1, alpha_2 = shortest_path_1(n_points, n, x_0_max)
    alpha_1_analytical, alpha_2_analytical = snellius_law(n)

    fig, ax = plt.subplots()
    ax.plot(alpha_1, alpha_2, linestyle="None",
            marker="o", markersize=10)
    ax.plot(alpha_1_analytical, alpha_2_analytical)
    ax.set_xlabel(r"$\alpha_1$")
    ax.set_ylabel(r"$\alpha_2$")

## Lichtweg durch ein Medium mit höhenabhängigem Brechungsindex als Extremalproblem

### Exponentiell abfallender Brechungsindex

Wir betrachten im Folgenden nur den oberen Halbraum, in dem der Brechungsindex ausgehend vom Wert `n_0` exponentiell als Funktion von $y$ abfallen soll. Die Funktion `n_np` berechnet sowohl den Brechungsindex `n` als auch seine Ableitung `nprime` an der Stelle `y`.

In [ ]:
def n_np(y, n_0):
    n = 1 + (n_0-1)*np.exp(-y)
    nprime = 1 - n
    return n, nprime

### Berechnung der optischen Weglänge

Um das Optimierungsproblem numerisch behandeln zu können, muss der Verlauf des Lichtstrahls diskretisiert werden. Dabei werden die Koordinaten der Punkte, die von `optimize.minimize` als eindimensionaler Vektor übergeben werden, zwischen den Randpunkten als variabel angesehen. Zur besseren Handhabung wird der Vektor `x` zunächst in ein zweidimensionales Array `xy` umgewandelt, das in der Achse 1 die $x$- und $y$-Koordinate enthält. Für jedes Wegstück wird zur Berechnung der optischen Weglänge der gemittelte Brechungsindex zwischen den jeweiligen Endpunkten verwendet. In `pathlens` sind die Längen der jeweiligen Wegstücke enthalten und die optische Länge `optical_length` ergibt sich daraus als mit dem Brechungsindex gewichtete Summe.

In [ ]:
def optical_path_length_2(x, n_0):
    xy = x.reshape(-1, 2)
    n, nprime = n_np(xy[:, 1], n_0)
    n_average = (n[1:] + n[:-1])/2
    pathlens = LA.norm(np.diff(xy, axis=0), axis=1)
    optical_length = np.sum(n_average * pathlens)
    return optical_length

### Minimierung der optischen Weglänge

Das zu lösende Problem ist durch eine Optimierungsaufgabe mit Randbedingungen gegeben. Insgesamt werden `n_points`+2 Punkte in der $x$-$y$-Ebene so angeordnet, dass die optische Weglänge minimal wird. Dabei muss aber sichergestellt werden, dass die beiden äußersten Punkte durch die beiden NumPy-Arrays `x_i` und `x_f` gegeben sind. Diese Randbedingung ist im Dictionary `x_bounds` definiert, das verlangt, dass die Funktion `boundaryvalues` den Wert 0 ergibt. Vor dem Beginn der Optimierung wird eine Startkonfiguration `x_init` vorgegeben, die Anfangs- und Endpunkt durch eine Gerade verbindet. Dabei muss das zunächst zweidimensionale Array mit Hilfe der `ravel`-Methode zur Verwendung in `optimize.minimize` in ein eindimensionales Array umgewandelt werden. Im Rahmen der Optimierung wird dieser gerade Verlauf verändert, wobei die Zahl der Iterationen auf `maxiter` beschränkt ist. Durch Setzen der Option `"disp"` auf `True` erreicht man die Ausgabe von Informationen über die Konvergenz des Optimierungsverfahrens. Die Koordinaten des berechneten Verlaufs sind im eindimensionalen Array `opt_result.x` enthalten, so dass wir für die spätere Darstellung $x$- und $y$-Koordinaten durch geeignetes Slicing separieren.

In [ ]:
def boundaryvalues(x, x_i, x_f):
    return LA.norm(x_i-x[:2]) + LA.norm(x_f-x[-2:])

In [ ]:
def shortest_path_2(n_0, x_i, x_f, n_points, maxiter):
    x_init = np.linspace(x_i, x_f, n_points+2).ravel()
    x_bounds = {"type": "eq",
                "fun": boundaryvalues,
                "args": (x_i, x_f)
                }
    opt_result = optimize.minimize(
        optical_path_length_2, x_init, args=(n_0,),
        constraints=x_bounds,
        options={"maxiter": maxiter, "disp": True})
    return opt_result.x[::2], opt_result.x[1::2]

### Implementierung der Bedienelemente und graphische Darstellung des optimierten Wegs

Mit Hilfe der Schieberegler können die folgenden Parameter eingestellt werden:
- `n_0`: Brechungsindex bei $y=0$
- `n_points`: Anzahl der variierbaren Punkte zwischen den Randpunkten
- `x_0`: $x$-Koordinate des Endpunkts
- `y_0`: $y$-Koordinate des Endpunkts
- `maxiter`: Maximale Anzahl der Iterationsschritte

Das Dictionary, das die Widgets definiert, wird hier durch eine Funktion zurückgegeben. Damit lassen sich die definierten Schieberegler auch im dritten Teil des Jupyter-Notebooks verwenden, ohne dass es zu einer Kopplung kommt.

In [ ]:
def get_widgets():
    return {"n_0":
            widgets.FloatSlider(
                value=1.4, min=1, max=2, step=0.01,
                description="$n(0)$"),
            "n_points":
            widgets.IntSlider(
                value=100, min=10, max=400, step=10,
                description="$M$"),
            "x_0":
            widgets.FloatSlider(
                value=2, min=0, max=5, step=0.01,
                description="$x_0$"),
            "y_0":
            widgets.FloatSlider(
                value=0.2, min=0, max=5, step=0.01,
                description="$y_0$"),
            "maxiter":
            widgets.IntSlider(
                value=200, min=100, max=2000, step=100,
                description="maxiter")
            }

In [ ]:
interact_start = interact_manual.options(
    manual_name="Start Berechnung")

@interact_start(**get_widgets())
def plot_result_gradient_1(n_0, n_points, x_0, y_0,
                           maxiter):
    x_i = np.array([0, 0])
    x_f = np.array([x_0, y_0])

    x, y = shortest_path_2(n_0, x_i, x_f, n_points,
                           maxiter)

    fig, ax = plt.subplots()
    ax.plot(x, y)
    ax.set_xlabel("$x$")
    ax.set_ylabel("$y$")

## Lichtweg durch ein Medium mit höhenabhängigem Brechungsindex als Randwertproblem

Nun wird als Alternative zur Lösung eines Optimierungsproblems der Verlauf des Lichtstrahls durch Lösung der eingangs angegebenen Euler-Lagrange-Gleichung bestimmt. Da die Randpunkte vorgegeben sind, handelt es sich hier um ein Randwertproblem.

### Definition der Differentialgleichung

In [ ]:
def dy_dx(x, y_vec, n_0):
    y, dydx = y_vec
    n, nprime = n_np(y, n_0)
    d2ydx2 = nprime * (1+dydx**2) / n
    return dydx, d2ydx2

### Randbedingungen

Die gesuchte Lösung soll $y(0)=0$ und $y(x_0)=y_0$ erfüllen.

In [ ]:
def bc(y_vec_a, y_vec_b, y_0):
    y_a, dydx_a = y_vec_a
    y_b, dydx_b = y_vec_b
    return y_a, y_b-y_0

### Berechnung der Lösung der Differentialgleichung

Als Anfangslösung wird mit Hilfe der beiden eindimensionalen Arrays `x_init` und `y_init` ein linearer Verlauf zwischen den Endpunkten definiert. Die zugehörigen Ableitungen $\text{d}y/\text{d}x$ sind in dem konstanten eindimensionalen Array `dy_dx_init` abgespeichert. Da `integrate.solve_bvp`, das zur Lösung des Randwertproblems verwendet wird, keine Möglichkeit zur Übergabe von Argumenten bietet, verwenden wir `partial`, um die Funktionen `dy_dx_without_n` und `bc_without_y0` zu definieren, in denen die Argumente `n_0` und `y_0` bereits festgelegt sind.

In [ ]:
def optical_path(x_0, y_0, n_points, n_0):
    x_init = np.linspace(0, x_0, n_points)
    y_init = np.linspace(0, y_0, n_points)
    dy_dx_init = np.full_like(x_init, y_0/x_0)
    dy_dx_without_n = partial(dy_dx, n_0=n_0)
    bc_without_y0 = partial(bc, y_0=y_0)
    solution = integrate.solve_bvp(
        dy_dx_without_n, bc_without_y0, x_init,
        [y_init, dy_dx_init])
    print(solution.message)
    return solution.x, solution.y[0]

### Implementierung der Bedienelemente und graphische Darstellung des optimierten Wegs

Die Bedeutung der Schieberegler ist weiter oben im Zusammenhang mit der Funktion `plot_result_gradient_2` beschrieben.

In [ ]:
@interact(**get_widgets())
def plot_result_gradient_2(n_0, n_points, x_0, y_0):
    x, y = optical_path(x_0, y_0, n_points, n_0)

    fig, ax = plt.subplots()
    ax.plot(x, y)
    ax.set_xlabel("$x$")
    ax.set_ylabel("$y$")